In [1]:

import cplex
import numpy as np
import time
import pickle
import numba
from numba import jit
import zlib

In [2]:
def toBinary(list2):
    n2=''
    for i in range(n):
        if i in list2:
            n2+='1'
        else:
            n2+='0'
    return(zlib.compress(n2.encode(),2))

In [3]:
path="./datos/EColiCore/"

fileName=path+"reversibles.txt"
with open(fileName, 'rb') as filehandle:
    reversibles = pickle.load(filehandle)

fileName=path+"irreversibles.txt"
with open(fileName, 'rb') as filehandle:
    irreversibles = pickle.load(filehandle)
irreversibles=list(irreversibles)

fileName=path+"inverse.txt"
with open(fileName, 'rb') as filehandle:
    inverse = pickle.load(filehandle)
fileName=path+"A2.txt"
with open(fileName, 'rb') as filehandle:
    A2 = pickle.load(filehandle)
mm=len(A2)

fileName=path+"indices.txt"
with open(fileName, 'rb') as filehandle:
    indices = pickle.load(filehandle)
    
fileName=path+"toNonDecoupled.txt"
with open(fileName, 'rb') as filehandle:
    toNonDecoupled = pickle.load(filehandle)
    
mainIndex=indices[0]
n=indices[1]
m=indices[2]

In [4]:
def create(k=1):
    global problem0, problem1, problem2, method
    
    B=np.zeros(len(A2))
    num_decision_var = n
    num_constraints = len(A2)
    problem0 = cplex.Cplex() 
    problem0.set_log_stream(None)
    method=problem0.parameters.lpmethod.values.primal
    problem0.parameters.lpmethod.set(method)
    problem0.set_error_stream(None)
    problem0.set_warning_stream(None)
    problem0.set_results_stream(None)
    problem0.variables.add(names= ["x"+str(i) for i in range(num_decision_var)])
    for i in range(num_decision_var):
        problem0.variables.set_lower_bounds(i, 0.0)

    for i in range(num_constraints):
        problem0.linear_constraints.add(
            lin_expr= [cplex.SparsePair(ind= A2[i][0], val= A2[i][1])],
            rhs= [B[i]],
            names = ["c"+str(i)],
            senses = ["E"]
        )

    problem1=cplex.Cplex(problem0)
    problem1.set_log_stream(None)
    problem1.parameters.lpmethod.set(method)
    problem1.parameters.advance.set(0)
    problem1.set_error_stream(None)
    problem1.set_warning_stream(None)
    problem1.set_results_stream(None)
    problem1.linear_constraints.add(
              rhs= [1],
              names = ["cc1"],
              senses = ["E"]
          )
    problem1.linear_constraints.set_linear_components([("cc1", cplex.SparsePair(ind= [mainIndex], val= [k]))])

    problem2=cplex.Cplex(problem0)
    problem2.set_log_stream(None)
    problem2.parameters.lpmethod.set(method)
    problem2.set_error_stream(None)
    problem2.set_warning_stream(None)
    problem2.set_results_stream(None)
    problem2.linear_constraints.add(
              rhs= [1],
              names = ["cc1"],
              senses = ["E"]
          )
    problem2.linear_constraints.add(
              rhs= [0],
              names = ["cc2"],
              senses = ["E"]
          )

In [5]:
def newSolution(n,m,f, lista, B):
    st=time.time()
    if len(lista)==1:
        myProblem=cplex.Cplex(problem1)
        myProblem.set_log_stream(None)
        myProblem.parameters.lpmethod.set(method)
        myProblem.set_error_stream(None)
        myProblem.set_warning_stream(None)
        myProblem.set_results_stream(None)
        mm=len(A2)+len(lista)
        myProblem.linear_constraints.set_linear_components([("cc1", cplex.SparsePair(ind= lista[0][0], val= lista[0][1]))])
    else:
        myProblem=cplex.Cplex(problem2)
        myProblem.set_log_stream(None)
        myProblem.parameters.lpmethod.set(method)
        myProblem.set_error_stream(None)
        myProblem.set_warning_stream(None)
        myProblem.set_results_stream(None)
        mm=len(A2)+len(lista)
        myProblem.linear_constraints.set_linear_components([("cc1", cplex.SparsePair(ind= lista[0][0], val= lista[0][1]))])
        myProblem.linear_constraints.set_linear_components([("cc2", cplex.SparsePair(ind= lista[1][0], val= lista[1][1]))])
  
    myProblem.objective.set_linear(f)
    myProblem.solve()  
    status=myProblem.solution.get_status()
    supports3=set()
    if status==1:
        
        mode2=myProblem.solution.get_values()
        sop2=frozenset([j for j in range(n) if abs(mode2[j])>10**-10])
        sop2
    else:
        return []
    return sop2

In [6]:
def searchEFMs(n,m,f):
    global problem1

    
    problem1.objective.set_linear(f)


    problem1.solve()


    solution=problem1.solution
    status=solution.get_status()
    supports3=set()
    if status==1:
        st=time.time()
        basis=solution.basis.get_basis()[0]
        basics=[ i for i in range(len(basis)) if basis[i]==1]
        nonBasics=list(set(range(n)).difference(set(basics)))
        basicsT=np.array(basics)
        nonBasicsT=np.array(nonBasics)

        As=solution.advanced.binvacol(nonBasics)


        mode2=(solution.get_values())

        As2=np.array([(x) for x in As])

        B2=[mode2[int(basics[i])] for i in range(len(basics))]
        B2T=np.array(B2)
        Bs=[i for i in range(len(B2)) if B2[i]==0]
        BsT=np.array(Bs)
        l1=len(basics)
        l2=len(nonBasics)
        mode2T=np.array(mode2)    
        st=time.time()
        supports2=analyze(n,m,basicsT,nonBasicsT,As2,mode2T,B2T,BsT,l1,l2) 
        t=(time.time()-st)
        supports=set([toBinary(sop) for sop in supports2])
        supports3.update(supports)

    return [supports3,t]

In [7]:
@jit
def analyze(n,m,basics,nonBasics,As,mode2,B2,Bs,l1,l2):
    newSupports=list()
    for index in range(l2):
        j0=nonBasics[index]
        column=As[index][:l1]
        column2=As[index][l1:]
        found=False
        if len(column2)>0:
            if min(column2)<0 or max(column2)>0:
                found=True
        if not found:
            found2=False
            if max(column)>0:
                found2=True
            if found2:
                indices=[]
                valid=True
                for i in range(len(column)):
                    if column[i]>0 and B2[i]==0:
                        valid=False
                        break
                    else:
                        indices.append(i)
                if valid:
                    values=np.array([B2[i]/column[i] if column[i]>0 else 100000 for i in range(l1)])
                    
                    minimum=min(values)  
                    minimums=np.where(values==minimum)[0]

                    for i0 in minimums:
                        j1=basics[i0]
                        newSol=np.zeros(n)
                        newSol[j0]=minimum
                        for i in range(l1):
                            j=basics[i]
                            newSol[j] = mode2[j] - column[i]*newSol[j0]    
                        newSol[j1]=0
                        bas=list(basics)
                        bas.append(j0)
                        sop2=[j for j in bas if newSol[j]>10**-10]
                        newSupports.append(sop2)
    return newSupports

In [8]:
def step0():
    
    global start, EFMsTypesExamine, l, maximum, times, supports, initials
    start=time.time()

    EFMsTypesExamine=[]
    l=2001
    for i in range(l):
        type2=set()
        EFMsTypesExamine.append(type2)

    maximum=l-1
    times=dict()

    supports=set()
    initials=1000

In [9]:
def step1():
    global supports, EFMsTypesExamine, times, inverse
    st2=time.time()

    for i in range(n):
        A3=A2.copy()
        indices=[i]
        indices2=[]
        if i in inverse.keys():
            indices2=[inverse[i]]
        values=[]
        values2=[]
        for j in indices:
            values.append(1)
            values2.append(1)
        if len(indices2)==0:
            list0=[[indices,values]]
            B=[1]
        else:
            list0=[[indices,values],[indices2,values2]]
            B=[1,0]
        f=[]
        for j in range(n):
            f.append([j,np.random.rand()]) 

        sop=newSolution(n,m,f, list0, B)
        sopB=toBinary(sop)
        if len(sop)==2:
            supports.add(sopB)
            times[sopB]=1
            EFMsTypesExamine[0].add(sopB)
            if not list(sop)[0] in inverse.keys():
                inverse[list(sop)[0]]=list(sop)[1]
                inverse[list(sop)[1]]=list(sop)[0]


        if sopB in supports:
            k=times[sopB]
            EFMsTypesExamine[k-1].remove(sopB)
            times[sopB]+=1
            EFMsTypesExamine[k].add(sopB)

        if not sopB in supports:
            supports.add(sopB)
            times[sopB]=1
            found=False
            for i in range(1,l+1):
                if sopB in EFMsTypesExamine[l-i]:
                    EFMsTypesExamine[l-i].remove(sopB)
                    i2=(l-i+1)
                    if i2==l:
                        i2=l-1
                    EFMsTypesExamine[i2].add(sopB)
                    found=True
                    break
            if not found:
                EFMsTypesExamine[0].add(sopB)

In [10]:
def initialize(k=1,mas=False):
    create(k)
    step0()
    step1()
    if mas:
        step2()
    print(len(supports))

In [11]:
initialize()

88


In [19]:
initialize(1,False)

examined=set()

t=0

factor=200

numberEFMs=100000
totalTime=5*60

st=time.time()
timeCounter=0
counter=0


tt=0
while timeCounter<totalTime and len(supports)<numberEFMs:
    if counter>0 and counter%1000==0:
        print(counter,len(supports),time.time()-st,t)

    for i in range(maximum+1):
        i0=i
        found=False
        if len(EFMsTypesExamine[i0])>0:
            sopB=EFMsTypesExamine[i0].pop()
            sop2=zlib.decompress(sopB).decode()
            sop=[j for j in range(len(sop2)) if sop2[j]=='1']
            examined.add(sopB)
            found=True
            break


    f=[[j,0]  if j in sop else [j,factor*np.random.rand()] for j in range(n)]
    
    obtainedEFMs=searchEFMs(n,mm,f)
    newSupports=obtainedEFMs[0]

    newSupports1=(newSupports.intersection(supports)).difference(examined)
    newSupports2=newSupports.difference(supports)

    for sop in newSupports1:
        i=times[sop]
        EFMsTypesExamine[i-1].remove(sop)
        EFMsTypesExamine[i].add(sop)
        times[sop]=times[sop]+1

    for sop in newSupports2:
        times[sop]=1
    supports.update(newSupports2)
    EFMsTypesExamine[0].update(newSupports2)
    counter+=1
    

print("End",len(supports),time.time()-st,t)

89
1000 10999 4.336782217025757 0
2000 20775 8.7766695022583 0
3000 30020 13.196382284164429 0
4000 38453 17.703749656677246 0
5000 46286 22.2682523727417 0
6000 53480 26.866063356399536 0
7000 60067 31.502310276031494 0
8000 65816 36.048670530319214 0
9000 71231 40.66485095024109 0
10000 76278 45.342296838760376 0
11000 80733 49.980902433395386 0
12000 84501 54.65543293952942 0
13000 87888 59.33031344413757 0
14000 90851 64.01948261260986 0
15000 93507 68.75331115722656 0
16000 95686 73.44009804725647 0
17000 97253 78.15126848220825 0
18000 98395 82.94435453414917 0
19000 99169 87.7409987449646 0
20000 99712 92.60813927650452 0
End 100000 97.24254179000854 0


In [13]:
translate=True
if translate:
    translatedSupports=set()
    for sop in supports:
        sopD=(zlib.decompress(sop)).decode()
        sopD2=[j for j in range(len(sopD)) if sopD[j]=='1']
        sopT=frozenset([toNonDecoupled[j] for j in sopD2])
        translatedSupports.add(sopT)